In [1]:
# imports for the function and conversion
import os
from collections import defaultdict
import json

In [2]:
# Converts a given COCO formatted json to YOLO formatted labels in a particular labelsFolder
def convert_to_YOLO(jsonPath, labelsFolder):
    with open(jsonPath, 'r') as f:
        data = json.load(f)
    
    # Group annotations by image_id for faster lookup
    annotations_by_image = defaultdict(list)
    for annotation in data['annotations']:
        annotations_by_image[annotation['image_id']].append(annotation)
    
    # Process each image
    for image in data['images']:
        image_id = image['id']
        img_width, img_height = image['width'], image['height']
        file_name = os.path.splitext(image['file_name'])[0]  # Remove extension
    
        # Open annotation file only once
        with open(f"{labelsFolder}/{file_name}.txt", "w") as label_file:
            for annotation in annotations_by_image.get(image_id, []):  # Lookup in preprocessed dict
                category_id = int(annotation['category_id']) - 1
                x_min, y_min, box_width, box_height = annotation['bbox']
    
                # Convert to YOLO format (class x center y center box width box height)
                x_center = (x_min + box_width / 2) / img_width
                y_center = (y_min + box_height / 2) / img_height
                norm_width = box_width / img_width
                norm_height = box_height / img_height
    
                # Write to label file
                label_file.write(f"{category_id} {x_center} {y_center} {norm_width} {norm_height}\n")
    print("COCO annotations converted to YOLO format!")

In [5]:
# Load COCO annotations and convert them to Yolo
annotations_file = 'POPClean/val/Annotations/val.json'
output_dir = 'POPYolo/labels/val'

convert_to_YOLO(annotations_file, output_dir)

COCO annotations converted to YOLO format!


In [1]:
from ultralytics import YOLO

In [2]:
import torch
torch.cuda.empty_cache()

In [9]:
# Load the pretrained Yolov8 nano model.
model = YOLO("yolov8n.pt")

# Train the YOLOv8 model
model.train(
    data="POPYolo/POPYolo.yaml",     # Dataset path
    epochs=20,                       # Number of epochs to train
    imgsz=640,                       # Image size
    project="Try2",     
    name="yolov8Test_model",                               
    exist_ok=True,                                      
)

Ultralytics 8.3.78  Python-3.12.9 torch-2.5.1+cu124 CUDA:0 (NVIDIA GeForce GTX 1660 Ti, 6144MiB)
engine\trainer: task=detect, mode=train, model=yolov8n.pt, data=POPYolo/POPYolo.yaml, epochs=20, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=Try2, name=yolov8Test_model, exist_ok=True, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_co

train: Scanning C:\Users\crazy\OneDrive\Desktop\Coding\Projects\HackAI\POPYolo\labels\train.cache... 5548 images, 0 backgrounds, 0 corrupt: 100%|██████████| 5548/5548 [00:00<?, 
val: Scanning C:\Users\crazy\OneDrive\Desktop\Coding\Projects\HackAI\POPYolo\labels\val.cache... 2320 images, 0 backgrounds, 0 corrupt: 100%|██████████| 2320/2320 [00:00<?, ?it/


Plotting labels to Try2\yolov8Test_model\labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.002, momentum=0.9) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0005), 63 bias(decay=0.0)
Image sizes 640 train, 640 val
Using 8 dataloader workers
Logging results to Try2\yolov8Test_model
Starting training for 20 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


       1/20      3.62G      1.427      1.414     0.9506         71        640: 100%|██████████| 347/347 [01:41<00:00,  3.43it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 73/73 [00:20<00:00,  3.50it/s]


                   all       2320       6546      0.836      0.735      0.788      0.483

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...

                   all       2320       6546      0.892      0.793      0.856      0.609



20 epochs completed in 0.668 hours.
Optimizer stripped from Try2\yolov8Test_model\weights\last.pt, 6.2MB
Optimizer stripped from Try2\yolov8Test_model\weights\best.pt, 6.2MB

Validating Try2\yolov8Test_model\weights\best.pt...
Ultralytics 8.3.78  Python-3.12.9 torch-2.5.1+cu124 CUDA:0 (NVIDIA GeForce GTX 1660 Ti, 6144MiB)
Model summary (fused): 72 layers, 3,005,843 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 73/73 [00:21<00:00,  3.44it/s]


                   all       2320       6546      0.905      0.786      0.859      0.612
Speed: 0.2ms preprocess, 3.1ms inference, 0.0ms loss, 1.2ms postprocess per image
Results saved to Try2\yolov8Test_model


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x000002BF0BBC1CD0>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.0480

In [8]:

bestModel = YOLO("Results/yolov8Test_model/weights/best.pt")
metrics = bestModel.val(
    data = "POPYolo/POPYolo.yaml"
)



Ultralytics 8.3.78  Python-3.12.9 torch-2.5.1+cu124 CUDA:0 (NVIDIA GeForce GTX 1660 Ti, 6144MiB)
Model summary (fused): 72 layers, 3,005,843 parameters, 0 gradients, 8.1 GFLOPs


val: Scanning C:\Users\crazy\OneDrive\Desktop\Coding\Projects\HackAI\POPYolo\labels\val.cache... 2320 images, 0 backgrounds, 0 corrupt: 100%|██████████| 2320/2320 [00:00<?, ?it/
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 145/145 [00:18<00:00,  8.01it/s]


                   all       2320       6546       0.91      0.756      0.833       0.56
Speed: 0.1ms preprocess, 1.8ms inference, 0.0ms loss, 1.0ms postprocess per image
Results saved to runs\detect\val2


In [ ]:
print(metrics.box.map)  # map50-95
print(metrics.box.map50)  # map50
print(metrics.box.map75)  # map75
print(metrics.box.maps)  # a list contains map50-95 of each category

In [4]:
from glob import glob

testImages = glob("POPClean/test/JPEGImages/*.JPG")

In [5]:
import random
from PIL import Image

In [6]:
# Run inference on 'bus.jpg'
results = bestModel(random.sample(testImages, 15))  # results list

print(len(results))
# Visualize the results
for i, r in enumerate(results):
    # Plot results image
    im_bgr = r.plot()  # BGR-order numpy array
    im_rgb = Image.fromarray(im_bgr[..., ::-1])  # RGB-order PIL image

    # Show results to screen (in supported environments)
    #r.show()

    # Save results to disk
    im_rgb.save(f"Predicted/results1{i}.jpg")

print("Finished")


0: 352x416 2 persons, 4.0ms
1: 352x416 (no detections), 4.0ms
2: 352x416 2 persons, 4.0ms
3: 352x416 1 person, 4.0ms
4: 352x416 (no detections), 4.0ms
5: 352x416 1 person, 4.0ms
6: 352x416 2 persons, 4.0ms
7: 352x416 (no detections), 4.0ms
8: 352x416 1 person, 4.0ms
9: 352x416 1 person, 4.0ms
10: 352x416 (no detections), 4.0ms
11: 352x416 (no detections), 4.0ms
12: 352x416 2 persons, 4.0ms
13: 352x416 (no detections), 4.0ms
14: 352x416 (no detections), 4.0ms
Speed: 1.9ms preprocess, 4.0ms inference, 16.2ms postprocess per image at shape (1, 3, 352, 416)
15
Finished
